In [1]:
import nltk

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vladislavkruglikov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vladislavkruglikov/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
import pandas as pd

df = pd.read_csv('data/Questions.csv', encoding="ISO-8859-1",
                 usecols=['Id', 'Title', 'Body', 'CreationDate', 'Score'])

df

,Id,CreationDate,Score,Title,Body
0,80,2008-08-01T13:57:07Z,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,2008-08-01T14:41:24Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,2008-08-01T15:50:08Z,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2008-08-01T18:42:19Z,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,2008-08-01T23:22:08Z,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...
...,...,...,...,...,...
995,83260,2008-09-17T13:34:46Z,3,How can I get the unique values of an array in...,"<p>Say I've got this array:\nMyArray(0)=""aaa""\..."
996,83320,2008-09-17T13:40:17Z,13,What is the difference between TrueType fonts ...,<p>What is the difference between TrueType fon...
997,83410,2008-09-17T13:48:25Z,26,How do I call a SQL Server stored procedure fr...,<p>I have a large CSV file and I want to execu...
998,83640,2008-09-17T14:09:20Z,3,C++ does begin/end/rbegin/rend execute in cons...,<p>For data types such as std::set and std::ma...


# Remove stopwords

In [3]:
from nltk import word_tokenize

nltk.download('stopwords')
nltk.download("punkt")

stopwords = set(nltk.corpus.stopwords.words('english'))

def remove_stop_words(text):
    text = text.lower()
    text_tokens = word_tokenize(text)
    tokens_without_sw = [w for w in text_tokens if not w in stopwords and w.isalnum()]
    return ' '.join(tokens_without_sw)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vladislavkruglikov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vladislavkruglikov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Lemmatize

In [4]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def lemmatize(text):
    text = text.lower()
    text_tokens = word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(w) for w in text_tokens]
    return ' '.join(lemmatized_tokens)

# Title preprocessing

In [5]:
title = df['Title']

df['Title preprocessed'] = title.apply(remove_stop_words).apply(lemmatize)

# Body preprocessing

In [6]:
import re

body = df['Body']

def remove_spaces(text):
    return ' '.join(text.split())

def remove_code(text):
    # remove code part
    text = re.sub('<pre>.*<\/pre>', '', text)
    # remove tags
    text = re.sub('<a.*?>', '', text)
    text = re.sub('</a>', '', text)
    text = re.sub('<code>', '', text)
    text = re.sub('</code>', '', text)
    text = re.sub('<p>', '', text)
    text = re.sub('</p>', '', text)
    return text

df['Body preprocessed'] = body.apply(remove_spaces).apply(remove_code).apply(remove_stop_words).apply(lemmatize)

# Fit TF-IDF title vectorizer

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

import pickle

tfidf_title_vectorizer = TfidfVectorizer()

title_preprocessed = df['Title preprocessed']

tfidf_title_vectorizer.fit(title_preprocessed)

pickle.dump(tfidf_title_vectorizer, open('data/tfidf_title_vectorizer.sav', 'wb'))

# Extract keywords from TF-IDF title vectorizer

In [8]:
def get_top_tfidf_words(sentence, tfidf_vectorizer, n=5):
    terms = tfidf_vectorizer.get_feature_names()
    sums = tfidf_vectorizer.transform([sentence]).sum(axis=0)
    data = []
    for col, term in enumerate(terms):
        if sums[0, col] > 0:
            data.append((term, sums[0, col]))
    data.sort(key=lambda x: -x[1])
    return ' '.join([i[0] for i in data[:n]])

df['Title preprocessed top 10 keywords'] = title_preprocessed.apply(lambda text: get_top_tfidf_words(text, tfidf_title_vectorizer, n=5))

# Fit TF-IDF body vectorizer

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_body_vectorizer = TfidfVectorizer()

body_preprocessed = df['Body preprocessed']

tfidf_body_vectorizer.fit(body_preprocessed)

pickle.dump(tfidf_body_vectorizer, open('data/tfidf_body_vectorizer.sav', 'wb'))

# Extract keywords from TF-IDF body vectorizer

In [10]:
df['Body preprocessed top 30 keywords'] = body_preprocessed.apply(lambda text: get_top_tfidf_words(text, tfidf_body_vectorizer, n=30))

# Get date score

The newest questions have higher score

In [11]:
df['timestamp'] = pd.DatetimeIndex(df.CreationDate).asi8

date_min = df.timestamp.min()
date_max = df.timestamp.max()

def date_score(date):
    return 1 + (date - date_min) / (date_max - date_min)

df['Date score'] = df.timestamp.apply(date_score)

# Get Vote score

In [12]:
votes = df['Score']

votes_min = votes.min()
votes_max = votes.max()

def votes_score(votes):
    return 1 + (votes - votes_min) / (votes_max - votes_min)

df['Votes score'] = df['Score'].apply(votes_score)

# Calculate TF-IDF vectors for title preprocessed top 10 keywords

Will use this to calculate distance between query TF-IDF vector and database documents TF-IDF vectors to find the most similar

In [13]:
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

df['Title preprocessed top 10 keywords TF-IDF vector'] = df['Title preprocessed top 10 keywords'].apply(lambda x: tfidf_title_vectorizer.transform([x]))

def calculate_distance(vec1, vec2):
    cosine_similarities = cosine_similarity(vec1.reshape(1, -1), vec2.reshape(1, -1))
    return cosine_similarities[0][0]

In [14]:
# For example

query = "multiple query one sql"

calculate_distance(df['Title preprocessed top 10 keywords TF-IDF vector'][0], tfidf_title_vectorizer.transform([query]))

0.7634472276912615

# Calculate TF-IDF vectors for body preprocessed top 30 keywords

In [15]:
df['Body preprocessed top 30 keywords TF-IDF vector'] = df['Body preprocessed top 30 keywords'].apply(lambda x: tfidf_body_vectorizer.transform([x]))

# Build Inverted Index for title

In [16]:
from collections import defaultdict

def build_inverted_index(data: list) -> list:
    inverted_index = defaultdict(list)
    rows = len(data)
    for idx in range(1, rows):
        sample = df.iloc[idx]
        union = sample['Title']
        for word in set(union.split()):
            inverted_index[word].append(sample.Id)
    return inverted_index

title_inverted_index = build_inverted_index(df['Title preprocessed top 10 keywords'])
pickle.dump(title_inverted_index, open('data/title_inverted_index.sav', 'wb'))

# Build Inverted Index for body

In [17]:
body_inverted_index = build_inverted_index(df['Body preprocessed top 30 keywords'])
pickle.dump(body_inverted_index, open('data/body_inverted_index.sav', 'wb'))

# Clear not needed cols

In [18]:
df.drop(columns=['CreationDate', 'timestamp', 'Score', 'Title preprocessed', 'Body preprocessed',
                 'Title preprocessed top 10 keywords', 'Body preprocessed top 30 keywords'], inplace=True)

# Look at the final dataset

In [19]:
df

,Id,Title,Body,Date score,Votes score,Title preprocessed top 10 keywords TF-IDF vector,Body preprocessed top 30 keywords TF-IDF vector
0,80,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,1.000000,1.007471,"(0, 1525)\t0.545568960255437\n (0, 1252)\t0...","(0, 6000)\t0.18721767746160486\n (0, 5872)\..."
1,90,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,1.000654,1.040122,"(0, 1657)\t0.4645992913411124\n (0, 1634)\t...","(0, 5555)\t0.2984741322522839\n (0, 5459)\t..."
2,120,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,1.001669,1.006087,"(0, 1463)\t0.6453861598243792\n (0, 953)\t0...","(0, 6021)\t0.1945865463080687\n (0, 5969)\t..."
3,180,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,1.004212,1.014942,"(0, 1753)\t0.5672717530973374\n (0, 646)\t0...","(0, 5872)\t0.12920224898559082\n (0, 5407)\..."
4,260,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,1.008345,1.013835,"(0, 1404)\t0.5559771794705758\n (0, 648)\t0...","(0, 5856)\t0.1724361322794553\n (0, 5485)\t..."
...,...,...,...,...,...,...,...
995,83260,How can I get the unique values of an array in...,"<p>Say I've got this array:\nMyArray(0)=""aaa""\...",1.999268,1.001107,"(0, 1704)\t0.4837082224165316\n (0, 1672)\t...","(0, 5986)\t0.1200227629525307\n (0, 5740)\t..."
996,83320,What is the difference between TrueType fonts ...,<p>What is the difference between TrueType fon...,1.999349,1.003874,"(0, 1650)\t0.6381218024803684\n (0, 626)\t0...","(0, 5534)\t0.6963856017965144\n (0, 2138)\t..."
997,83410,How do I call a SQL Server stored procedure fr...,<p>I have a large CSV file and I want to execu...,1.999469,1.007471,"(0, 1536)\t0.4541596470287601\n (0, 1510)\t...","(0, 5872)\t0.15164468295753483\n (0, 5855)\..."
998,83640,C++ does begin/end/rbegin/rend execute in cons...,<p>For data types such as std::set and std::ma...,1.999778,1.001107,"(0, 1622)\t0.3978818809204913\n (0, 1529)\t...","(0, 5969)\t0.11671257573527914\n (0, 5564)\..."


# Save preprocessed dataset

In [20]:
pickle.dump(df, open('data/Preprocessed Questions.sav', 'wb'))